In [ ]:
!pip install ultralytics

In [2]:
import cv2
import os
from ultralytics import YOLO
import numpy as np

In [3]:
correct_classes = {
    0: 2, 
    1: 1, 
    2: 11, 
    3: 0, 
    4: 9, 
    5: 23, 
    6: 3, 
    7: 15, 
    8: 13, 
    9: 24, 
    10: 10, 
    11: 12, 
    12: 19, 
    13: 20, 
    14: 22,
    15: 16, 
    16: 14, 
    17: 7, 
    18: 6, 
    19: 5, 
    20: 21, 
    21: 8, 
    22: 18, 
    23: 17, 
    24: 4
    }

In [21]:

def process_image(image_path, lines, resize_to=(480, 480)):
    pred_labels_true = []
    pred_labels_all = []
    
    true_labels = []
    

    image = cv2.imread(image_path)
    
    for line in lines:
        true_labels.append(convert_from_yolo_format(line, image.shape[1], image.shape[0]))
    
    results = model_dt.predict(source=image, show= False, verbose=False )
    results[0] = results[0].cpu()
    results[0] = results[0].numpy()
    confession = results[0].boxes.conf.tolist()
    bboxes = results[0].boxes.xyxy.tolist()
    for _, box in enumerate(bboxes):
        x_min, y_min, x_max, y_max = box
        cropped_image = image[int(y_min):int(y_max), int(x_min):int(x_max)]
        cropped_image = cv2.resize(cropped_image, resize_to)
        resultss = model_cl.predict(source=cropped_image, show= False, verbose=False )
        classes = resultss[0].names
        probs = resultss[0].probs.top1
        results[0].boxes.cls[_] = correct_classes[probs]
        # pred_labels.append(convert_to_yolov5_format(int(results[0].boxes.cls[0]) ,box, image.shape[1], image.shape[0]))
        pred_labels_all.append([(x_min, y_min, x_max, y_max), int(results[0].boxes.cls[0]), confession[_]])
        for label in true_labels:
            if int(results[0].boxes.cls[0]) == label[1]:
                pred_labels_true.append([(x_min, y_min, x_max, y_max), int(results[0].boxes.cls[0]), confession[_]])

    if len(pred_labels_true) > 0:
        return results , pred_labels_true, true_labels
    else:
        return results , pred_labels_all, true_labels


def convert_to_yolo_format(cls, bbox, img_width, img_height):
    yolo_format = ''
    x1, y1, x2, y2 = bbox
    xc = (x1 + x2) / 2 / img_width
    yc = (y1 + y2) / 2 / img_height
    w = (x2 - x1) / img_width
    h = (y2 - y1) / img_height
    yolo_format += f'{cls} {xc} {yc} {w} {h}\n'
    return yolov_format
    
def convert_from_yolo_format(yolo_format, img_width, img_height):
    yolo_format = yolo_format.split()
    cls = yolo_format[0]
    xc = float(yolo_format[1]) * img_width
    yc = float(yolo_format[2]) * img_height
    w = float(yolo_format[3]) * img_width
    h = float(yolo_format[4]) * img_height
    x1 = xc - w / 2
    y1 = yc - h / 2
    x2 = xc + w / 2
    y2 = yc + h / 2
    return [(x1, y1, x2, y2), int(cls)]

def calculate_iou(box1, box2):
    intersection_x = max(0, min(box1[2], box2[2]) - max(box1[0], box2[0]))
    intersection_y = max(0, min(box1[3], box2[3]) - max(box1[1], box2[1]))
    intersection_area = intersection_x * intersection_y

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - intersection_area

    if union_area == 0:
        return 0

    iou = intersection_area / union_area
    return iou

def calculate_recall(true_positives, total_positives):
    if total_positives == 0:
        return 0
    return true_positives / total_positives

def calculate_precision(true_positives, false_positives):
    if true_positives + false_positives == 0:
        return 0
    return true_positives / (true_positives + false_positives)

def calculate_average_precision(recalls, precisions):
    sorted_indices = np.argsort(np.array(recalls))
    sorted_recalls = np.array(recalls)[sorted_indices]
    sorted_precisions = np.array(precisions)[sorted_indices]

    area_under_curve = 0
    max_precision = 0
  
    for i in range(1, len(sorted_recalls)):
        max_precision = max(max_precision, sorted_precisions[i])
        area_under_curve += (sorted_recalls[i] - sorted_recalls[i-1]) * max_precision
    return area_under_curve

def calculate_map50(average_precisions):
    if len(average_precisions) == 0:
        return 0
    return np.mean(average_precisions)

In [23]:


model_dt = YOLO("/kaggle/input/detect/best.pt")
model_cl = YOLO("/kaggle/input/classific/best (3).pt")

NUM_CLASSES = len(correct_classes)

# Создаем списки для хранения данных
ious = []
recalls = [[] for _ in range(NUM_CLASSES)]
precisions = [[] for _ in range(NUM_CLASSES)]
average_precisions = [[] for _ in range(NUM_CLASSES)]

folder_path = "/kaggle/input/dataset/Dataset_0/val/images"
pred_labels_dict = []
fact_labels_dict = []


for i, filename in enumerate(os.listdir(folder_path)):
    print(filename.split('.')[0])
    image_path = os.path.join(folder_path, filename)
    label_path = '/kaggle/input/labelss/labels_1/' + str(filename.split('.')[0]) + ".txt"
    
    if os.path.isfile(image_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        with open(label_path, 'r') as file:
            lines = file.readlines()
            prediction, predicted_boxes, true_boxes = process_image(image_path, lines)
            pred_labels_dict.append(predicted_boxes)
            fact_labels_dict.append(true_boxes)

        for true_box in true_boxes:
            true_class = true_box[1]
            iou_scores = []
            
            for predicted_box in predicted_boxes:
                predicted_class = predicted_box[1]
                iou = calculate_iou(true_box[0], predicted_box[0])
                
                if predicted_class == true_class:
                    iou_scores.append(iou)
        
            if len(iou_scores) > 0:
                best_iou = max(iou_scores)
                ious.append(best_iou)

        # Рассчитываем Recall, Precision и Average Precision для каждого класса
        for c in range(NUM_CLASSES):
            true_positives = 0
            false_positives = 0
            total_positives = 0
            
            for true_box in true_boxes:
                if true_box[1] == c:
                    total_positives += 1
            
            for predicted_box in predicted_boxes:
                if predicted_box[1] == c:
                    if predicted_box[2] >= 0.5:
                        total_positives += 1
                        
                        iou_scores = []
                        for true_box in true_boxes:
                            if true_box[1] == c:
                                iou = calculate_iou(true_box[0], predicted_box[0])
                                iou_scores.append(iou)
                                
                        if len(iou_scores) > 0 and max(iou_scores) >= 0.5: # Порог IoU равен 0.5
                            true_positives += 1
                        else:
                            false_positives += 1
            
            recalls[c].append(calculate_recall(true_positives, total_positives))
            precisions[c].append(calculate_precision(true_positives, false_positives))
            average_precisions[c].append(calculate_average_precision(recalls[c], precisions[c]))


In [24]:
a = []
for _, i in enumerate(average_precisions):
    map50 = calculate_map50(i)
    print("MAP@50 ", str(_), ": ", map50)
    a.append(map50)
map50 = calculate_map50(a)
print("MAP@50 all: ", map50)

MAP@50  0 :  0.34721533630804824
MAP@50  1 :  0.4429433151545199
MAP@50  2 :  0.3528755577590481
MAP@50  3 :  0.38617583870434635
MAP@50  4 :  0.32353123450669313
MAP@50  5 :  0.4921500578416791
MAP@50  6 :  0.4659147248388696
MAP@50  7 :  0.36440257808626675
MAP@50  8 :  0.3273839034870269
MAP@50  9 :  0.36936043629152204
MAP@50  10 :  0.4225300225857984
MAP@50  11 :  0.3040881672450834
MAP@50  12 :  0.40282597917699553
MAP@50  13 :  0.3304928937365725
MAP@50  14 :  0.333002809452983
MAP@50  15 :  0.3303586184101801
MAP@50  16 :  0.3666336142786316
MAP@50  17 :  0.3180466038671294
MAP@50  18 :  0.3542389687654933
MAP@50  19 :  0.37584696744339774
MAP@50  20 :  0.492563212692117
MAP@50  21 :  0.3004358783672121
MAP@50  22 :  0.3439100975045447
MAP@50  23 :  0.3586597256651793
MAP@50  24 :  0.3513882002974715
MAP@50 all:  0.3702789896986724
